In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from adam_prepare import titanic_pipeline

## 1. Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [2]:
train, val, test = titanic_pipeline()
train.shape, val.shape, test.shape

found data


((623, 10), (134, 10), (134, 10))

______

In [3]:
X_train = train.drop(columns = ['survived', 'deck'])
y_train = train.survived

X_val = val.drop(columns = ['survived', 'deck'])
y_val = val.survived

------

In [4]:
X_train = pd.get_dummies(X_train, columns = ['sex'], drop_first = True)
X_train = pd.get_dummies(X_train, columns = ['class', 'embark_town'])

X_val = pd.get_dummies(X_val, columns = ['sex'], drop_first = True)
X_val = pd.get_dummies(X_val, columns = ['class', 'embark_town'])



In [5]:
X_train.shape, X_val.shape

((623, 12), (134, 12))

---

In [6]:
X_train.head()

,age,sibsp,parch,fare,alone,sex_male,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
748,19.0,1,0,53.1000,0,1,1,0,0,0,0,1
45,29.0,0,0,8.0500,1,1,0,0,1,0,0,1
28,29.0,0,0,7.8792,1,0,0,0,1,0,1,0
633,29.0,0,0,0.0000,1,1,1,0,0,0,0,1
403,28.0,1,0,15.8500,0,1,0,0,1,0,0,1


---

In [9]:
seed = 42

rf = RandomForestClassifier(min_samples_leaf = 1, max_depth = 10, random_state = seed)

---

In [10]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=42)

---

In [14]:
fi = pd.DataFrame({'feature': X_train.columns,
                  'importance': rf.feature_importances_})

fi.sort_values(by = 'importance', ascending = False)

,feature,importance
5,sex_male,0.304415
3,fare,0.227986
0,age,0.208397
8,class_Third,0.059482
2,parch,0.044518
1,sibsp,0.042962
6,class_First,0.033793
4,alone,0.021228
11,embark_town_Southampton,0.018501
7,class_Second,0.015583


In [17]:
y_pred = rf.predict(X_train)

y_pred[:5]

array([0, 0, 1, 0, 0])

In [18]:
y_pred_proba = rf.predict_proba(X_train)

y_pred_proba

array([[0.63485783, 0.36514217],
       [0.92536919, 0.07463081],
       [0.05132852, 0.94867148],
       ...,
       [0.94831313, 0.05168687],
       [0.9052349 , 0.0947651 ],
       [0.01368421, 0.98631579]])

## 2. Evaluate your results using the model score, confusion matrix, and classification report.

In [11]:
rf.score(X_train, y_train)

0.9550561797752809

---

In [12]:
rf.score(X_val, y_val)

0.8432835820895522

---

## 3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

## 4. Run through steps increasing your min_samples_leaf and decreasing your max_depth.

## 5. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

## After making a few models, which one has the best performance (or closest metrics) on both train and validate?